## Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

### Load and process data

In [ ]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()
cur_path = cur_path + '# path to training set directory'

for i in range(classes):
    path = os.path.join(cur_path, str(i))
    images = os.listdir(path)
    
    for img in images:
        try:
            image = Image.open(path + '/' + img)
            image = np.array(image.resize((30, 30)))
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image...")
            
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

### Train-Test Split

In [ ]:
def split_data(data, labels):
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=100)

    y_train = to_categorical(y_train, 43)
    y_test = to_categorical(y_test, 43)

    print(x_train.shape, y_train.shape)
    print(x_test.shape, y_test.shape)
    
    return x_train, x_test, y_train, y_test

## Deep Learning Model

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=x_train.shape[1:]))
    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

### Train Model

In [ ]:
def train(x_train, x_test, y_train, y_test):
    model = create_model()
    history = model.fit(x_train, y_train, 
                        batch_size=64, 
                        epochs=15, 
                        validation_data=(x_test, y_test))
    return history, model

In [ ]:
def create_plots(history):
    plt.subplot(1, 2, 1)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='Training Loss')
    plt.plot(history.history['val_loss'], color='orange', label='Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], color='orange', label='Validation Accuracy')
    plt.legend()

    plt.show()

In [ ]:
def test(model):
    
    path = os.getcwd() + '#path'
    test_y = pd.read_csv(path + '/Test.csv')
    labels = test_y['ClassId'].values
    imgs = test_y['Path'].values

    test_imgs = []
    
    for img in imgs:
        image = Image.open(path + '/' + img)
        image = np.array(image.resize((30, 30)))
        test_imgs.append(image)
    
    test_imgs = np.array(test_imgs)
    
    pred = np.argmax(model.predict(test_imgs), axis=1)
    acc = accuracy_score(labels, pred)
    
    return acc

Achieved Test Accuracy: 96.199%

In [ ]:
%%time

x_train, x_test, y_train, y_test = split_data(data, labels)
history, model = train(x_train, x_test, y_train, y_test)
create_plots(history)
acc = test(model)
print(acc)